<a href="https://colab.research.google.com/github/SorahP/ddPn08_RVC-WebUI-colab/blob/main/ddPn08_RVC_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title GPU確認
!nvidia-smi

In [ ]:
#@title ライブラリをインストール
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2

In [ ]:
#@title rvc-webuiをダウンロード

!git clone https://github.com/ddPn08/rvc-webui.git
%cd /content/rvc-webui/
!mkdir -p pretrained uvr5_weights
!git pull

In [ ]:
#@title aria2をインストール ＆ 必要なファイルをダウンロード
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/rvc-webui/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/rvc-webui/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/rvc-webui/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/rvc-webui/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/rvc-webui/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/rvc-webui/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/rvc-webui/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/rvc-webui/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/rvc-webui/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/rvc-webui/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/rvc-webui/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/rvc-webui/pretrained -o f0G48k.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/rvc-webui/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/rvc-webui/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

In [ ]:
#@title hubert_baseをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/rvc-webui -o hubert_base.pt

In [ ]:
#@title ドライブをマウント

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title データセットを展開

#@markdown データセットのフルパス
DATASET = "/content/drive/MyDrive/RVC-1/Dataset1.zip"  #@param {type:"string"}

!mkdir -p /content/dataset
!unzip -d /content/dataset -B {DATASET}

In [ ]:
#@title ファイルのリネーム
!ls -a /content/dataset/
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/*.*~*

In [ ]:
#@title 重みをドライブからダウンロード
#@markdown Discriminatorのパス
discriminator = "/content/drive/MyDrive/RVC-1/f0G48k768_jphubert_tuned.pth"  #@param {type:"string"}
#@markdown Generatorのパス
generator = "/content/drive/MyDrive/RVC-1/f0D48k768_jphubert_tuned.pth"  #@param {type:"string"}
!cp {discriminator} /content/rvc-webui/pretrained/
!cp {generator} /content/rvc-webui/pretrained/f0G48k768_jphubert_tuned.pth

In [ ]:
#@title WebUIを起動
%cd /content/rvc-webui
!python3 launch.py --share --colab --pycmd python3